# Problema2 - REALIZAR SOBRE ARCHIVO .PY  (NO JUPYTER) 


Lea el archivo 'winemag-data-130k-v2.csv' y realice lo siguiente:

- Explore el dataframe según lo visto en clase
- Realice al menos  4 renombre de columna y 3 creación de nuevas columnas según criterio. Deberá crear las columnas que crea conveniente. 
    Ejemplo: Según el país etiquetelos según continente.

- Genere 4 reportes distintos(podria agrupando, filtrar, ordenar, etc). Deberá elegir que reportes realizar
    - Ejemplo: Por contienente mostrar los vinos mejor puntuados
    - Ejemplo2: Promedio de precio de vino y cantidad de reviews obtenidos según pais. Ordenado de mejor a peor.



- Exporte los 4 reportes a 4 formatos de archivos diversos, csv, excel, sqllite, mongodb...
   
   Para guardar datos de datos agrupados, puede revisar esta solución https://stackoverflow.com/questions/25789264/pandas-dataframegroupby-export-to-excel

- Envie uno de los reportes generados por correo, mostrar un pantallazo del correo enviado




### Datos 

winemag-data-130k-v2.csv
- https://github.com/gdelgador/ProgramacionPython202401/blob/main/Modulo5/src/winemag-data-130k-v2.csv

País según continente:
- https://gist.githubusercontent.com/kintero/7d1db891401f56256c79/raw/a61f6d0dda82c3f04d2e6e76c3870552ef6cf0c6/paises.csv
 






In [ ]:
import pandas as pd
import sqlite3
from pymongo import MongoClient
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

# 1. Cargar el archivo CSV
df = pd.read_csv('winemag-data-130k-v2.csv')

# 2. Exploración del DataFrame
print(df.head())  # Mostrar primeras filas
print(df.info())  # Información de las columnas
print(df.describe())  # Estadísticas numéricas

# 3. Renombrar columnas y crear nuevas columnas
df.rename(columns={'points': 'puntos', 'price': 'precio', 'country': 'pais', 'variety': 'variedad'}, inplace=True)

# Crear nuevas columnas
df['rango_precios'] = pd.cut(df['precio'], bins=[0, 20, 50, 100, 200, 1000], labels=['Barato', 'Accesible', 'Moderado', 'Caro', 'Muy Caro'])
df['longitud_nombre'] = df['title'].apply(len)
df['continente'] = df['pais'].map({
    'France': 'Europa',
    'Italy': 'Europa',
    'US': 'América',
    'Argentina': 'América',
    'Chile': 'América',
})

print(df.head())

# 4. Generar los reportes

# Reporte 1: Mejores vinos por continente según puntaje
reporte_1 = df.groupby('continente').apply(lambda x: x.nlargest(5, 'puntos'))[['title', 'puntos']]
print(reporte_1)

# Reporte 2: Promedio de precio de vino y cantidad de reviews por país
reporte_2 = df.groupby('pais').agg({
    'precio': 'mean',
    'puntos': 'count'
}).rename(columns={'puntos': 'cantidad_reviews'}).sort_values(by='precio', ascending=False)
print(reporte_2)

# Reporte 3: Vinos más baratos de cada variedad
reporte_3 = df.loc[df.groupby('variedad')['precio'].idxmin()][['title', 'variedad', 'precio']]
print(reporte_3)

# Reporte 4: Distribución de rangos de precios por país
reporte_4 = df.groupby(['pais', 'rango_precios']).size().unstack(fill_value=0)
print(reporte_4)

# 5. Exportar reportes a diferentes formatos

# Exportar reporte 1 a CSV
reporte_1.to_csv('reporte_mejores_vinos.csv', index=False)

# Exportar reporte 2 a Excel
reporte_2.to_excel('reporte_precio_reviews.xlsx', index=True)

# Exportar reporte 3 a SQLite
conn = sqlite3.connect('reportes_vinos.db')
reporte_3.to_sql('vinos_baratos', conn, if_exists='replace', index=False)
conn.close()

# Exportar reporte 4 a MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['winemag']
collection = db['reporte_precios']
data_dict = reporte_4.reset_index().to_dict(orient='records')
collection.insert_many(data_dict)

# 6. Enviar reporte por correo
def enviar_email(destinatario, asunto, cuerpo, archivo):
    remitente = 'tu_email@gmail.com'
    password = 'tu_contraseña'
    
    msg = MIMEMultipart()
    msg['From'] = remitente
    msg['To'] = destinatario
    msg['Subject'] = asunto

    msg.attach(MIMEText(cuerpo, 'plain'))
    attachment = open(archivo, 'rb')
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f"attachment; filename= {archivo}")
    msg.attach(part)
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(remitente, password)
    text = msg.as_string()
    server.sendmail(remitente, destinatario, text)
    server.quit()
enviar_email('destinatario@example.com', 'Reporte de Vinos', 'Adjunto se encuentra el reporte solicitado.', 'reporte_mejores_vinos.csv')



